In [1]:
import time 

from IPython.core.display import display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.style.use('ggplot')
import numpy as np
import scipy
import seaborn as sns
sns.set(font='Osaka')
import pandas as pd

In [2]:
t0 = time.time()
data_dir = '../data/'
file = 'df_item_transaction.csv'
df_item_transaction = pd.read_csv(data_dir + file)
print('Load in ', time.time() - t0)

Load in  1.3073570728302002


In [3]:
country = 'United Kingdom'
df_uk = \
df_item_transaction[df_item_transaction['Country'] == country].drop('Unnamed: 0', axis=1)

# CustomerID to Nominal
df_uk['CustomerID'] = df_uk['CustomerID'].fillna('NaN')

# InvoiceDate to Datetime
df_uk['InvoiceDate'] = pd.to_datetime(df_uk['InvoiceDate'])

assert df_uk['StockCode'].nunique() == 3910
assert df_uk['CustomerID'].nunique() == 3918

print('# of distinct items =', df_uk['StockCode'].nunique())
print('# of distinct customers =', df_uk['CustomerID'].nunique())

# of distinct items = 3910
# of distinct customers = 3918


In [4]:
# Customer ごとにデータを集計　
# Recency
# Frequency = #ofTransactions
# Monetary = sum(Sales)

# Sales: min, max, median, mean, sum
# Quantity: min, max, median, mean, sum
# Quantity / Transaction: min, max, median, mean
# Unitprice: min, max, median, mean

# Item / Transaction = 商品の種類 / Transaction: min, max, median, mean


# 各商品をいくつ購入したか？　(3,910 items hence the same # of columns)

# TODO:
# ある商品を購入する顧客は Frequency が大きいといった関係が発見できるか？
# RMF 分析で優良顧客を見つけ出し、彼らの 2月, 4月の


# Monetary
df_uk_per_customer = \
pd.DataFrame(df_uk.groupby('CustomerID')['Sales'].sum()).reset_index()\
.rename(columns={'Sales': 'Monetary'})

# Frequency
df_uk_per_customer['Frequency'] = \
pd.DataFrame(df_uk.groupby('CustomerID')['InvoiceNo'].nunique()).reset_index()['InvoiceNo']
# small test
test_customer = 12748
assert df_uk_per_customer[df_uk_per_customer['CustomerID'] == test_customer]['Frequency'].values\
            == df_uk[df_uk['CustomerID'] == test_customer]['InvoiceNo'].nunique()

# Recency
# 基準日
df_uk_per_customer['ReferenceDate'] = \
    pd.to_datetime(df_item_transaction['InvoiceDate']).max()
# 最終購入日
df_uk_per_customer['LastTransactionDate'] = \
    pd.DataFrame(df_uk.groupby('CustomerID')['InvoiceDate'].max()).reset_index()['InvoiceDate']
# 基準日 - 最終購入日
df_uk_per_customer['Recency(Day)'] = \
(df_uk_per_customer['ReferenceDate'].dt.date - df_uk_per_customer['LastTransactionDate'].dt.date).dt.days


# sum(Quantity)
df_uk_per_customer['QuantitySum'] = pd.DataFrame(df_uk.groupby('CustomerID')['Quantity'].sum()).reset_index()['Quantity']

# Quantity / Transaction
df_uk_per_customer['QuantityPerTransaction'] = df_uk_per_customer['QuantitySum'] / df_uk_per_customer['Frequency']

# Sales / Transaction
df_uk_per_customer['Sales/Transaction'] = df_uk_per_customer['Monetary'] / df_uk_per_customer['Frequency']

In [5]:
display(df_uk_per_customer.head(3))

,CustomerID,Monetary,Frequency,ReferenceDate,LastTransactionDate,Recency(Day),QuantitySum,QuantityPerTransaction,Sales/Transaction
0,12346,77183.60,1,2011-12-09 12:50:00,2011-01-18 10:01:00,325,74215,74215.000000,77183.600000
1,12747,4196.01,11,2011-12-09 12:50:00,2011-12-07 14:34:00,2,1275,115.909091,381.455455
2,12748,32317.32,206,2011-12-09 12:50:00,2011-12-09 12:20:00,0,25511,123.839806,156.880194


In [27]:
# User-Item matrix

# CustomerID = NaN を除外
df_uk = df_uk[df_uk['CustomerID'] != 'NaN']

df_uk_user_item_matrix = \
pd.DataFrame(df_uk.groupby(['CustomerID', 'StockCode'])['Quantity'].sum())\
.reset_index()\
.pivot(index='CustomerID', columns='StockCode', values='Quantity')\
.fillna(0)

In [34]:
df_uk_user_item_matrix.head(3)

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,PADS
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12747.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12748.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,28.0,36.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## KMeans clustering

In [29]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
    
n_clusters_candidates = [2, 3, 4, 5]

for n_clusters in n_clusters_candidates:
    clusterer = KMeans(n_clusters=n_clusters, random_state=0)
    clusterer.fit(df_uk_user_item_matrix)
    
    labels = clusterer.labels_
    score = silhouette_score(df_uk_user_item_matrix, labels, metric='manhattan')
    print('n_clusters: ', n_clusters, '\tsilhouette_score: ', score)

n_clusters:  2 	silhouette_score:  0.97542691416
n_clusters:  3 	silhouette_score:  0.973433069436
n_clusters:  4 	silhouette_score:  0.933384376444
n_clusters:  5 	silhouette_score:  0.939179245421


In [19]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
    
n_clusters_candidates = [6, 7, 8, 9]

for n_clusters in n_clusters_candidates:
    clusterer = KMeans(n_clusters=n_clusters, random_state=0)
    clusterer.fit(df_uk_user_item_matrix)
    
    labels = clusterer.labels_
    score = silhouette_score(df_uk_user_item_matrix, labels, metric='manhattan')
    print('n_clusters: ', n_clusters, '\tsilhouette_score: ', score)

n_clusters:  6 	silhouette_score:  0.93651770603
n_clusters:  7 	silhouette_score:  0.939429442255
n_clusters:  8 	silhouette_score:  0.937491270555
n_clusters:  9 	silhouette_score:  0.94006746434


In [30]:
from sklearn.cluster import KMeans

n_clusters = 2
clusterer = KMeans(n_clusters=n_clusters, random_state=0)
clusterer.fit(df_uk_user_item_matrix)
labels = clusterer.labels_

In [31]:
clusterer.cluster_centers_.shape

(2, 3641)

In [32]:
clusterer.cluster_centers_

array([[ 0.10240041,  0.07431052,  0.04673136, ...,  0.00306435,
         0.00306435,  0.00076609],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [33]:
clusterer.labels_

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [36]:
df_uk_user_item_matrix_2_clusters = df_uk_user_item_matrix.copy()

In [37]:
df_uk_user_item_matrix_2_clusters['Clusters'] = clusterer.labels_

In [42]:
df_uk_user_item_matrix_2_clusters.head(3)

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,PADS,Clusters
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
12747.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
12748.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,28.0,36.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [40]:
df_uk_user_item_matrix_2_clusters.groupby('Clusters').size()

Clusters
0    3916
1       1
dtype: int64

### 3 clusters

In [43]:
from sklearn.cluster import KMeans

n_clusters = 3
clusterer = KMeans(n_clusters=n_clusters, random_state=0)
clusterer.fit(df_uk_user_item_matrix)
labels = clusterer.labels_

In [44]:
df_uk_user_item_matrix_3_clusters = df_uk_user_item_matrix.copy()
df_uk_user_item_matrix_3_clusters['Clusters'] = clusterer.labels_

In [45]:
df_uk_user_item_matrix_3_clusters.groupby('Clusters').size()

Clusters
0    3915
1       1
2       1
dtype: int64

In [46]:
df_uk_user_item_matrix_3_clusters[df_uk_user_item_matrix_3_clusters['Clusters'] == 1]

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,PADS,Clusters
CustomerID,,,,,,,,,,,,,,,,,,,,,
16446.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
